In [2]:
import sys
import os
sys.path.append(os.path.abspath("../"))
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from utils.categorize_brand import categorize_brand

df = pd.read_csv("../data/sales_ads_train.csv")

 # buduje listę ilośći aut w widełkach co 50k zeby zobaczyć gdzie nalezy oczyścić dane
bins = list(range(0, int(df["Cena"].max()) + 50000, 50000)) 
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]
df["Widełki_cenowe"] = pd.cut(df["Cena"], bins=bins, labels=labels, include_lowest=True)
widełki_count = df["Widełki_cenowe"].value_counts().sort_index()

# ucinam ceny do 1000 (38 ofert) i od 2 mln praktycznie brak ofert
df_filtered = df[(df["Cena"] >= 1000) & (df["Cena"] <= 2000000)]

# usuwam zbędne kolumny - "Widełki_cenowe", "Data_publikacji_oferty", "ID"
df_filtered = df_filtered.drop(columns=["Widełki_cenowe", "Data_publikacji_oferty", "ID"])

# usuwam wieksze w których waluta != PLN oraz usuwam zbędną kolumnę
df_filtered = df_filtered[df_filtered["Waluta"] == "PLN"]
df_filtered = df_filtered.drop(columns=["Waluta"])

# usuwamy kolumny które mają duzo braków a potencjalnie mały wpływ na cenę
df_filtered = df_filtered.drop(columns=["Wersja_pojazdu", "Generacja_pojazdu", "Emisja_CO2","Data_pierwszej_rejestracji", "Lokalizacja_oferty"])

# Pierwszy właściciel ma tylko 2 wartości - [nan, 'Yes'] więc zamieniamy je na 0 i 1
df_filtered["Pierwszy_wlasciciel"] = df_filtered["Pierwszy_wlasciciel"].notna().astype(int)
# df_filtered.info()

# Kraj pochodzenia podnosi cenę tylko jeśli to jest Polska więc zamieniamy na 1 dla Polski i 0 dla reszty świata
df_filtered["Kraj_pochodzenia"] = (df_filtered["Kraj_pochodzenia"] == "Poland").astype(int)

# Stan jest ['Used', 'New', nan] więc nan traktujemy jako used
df_filtered["Stan"] = (df_filtered["Stan"] == "New").astype(int)

# Grupujemy marki na premium, srednie i budzetowe i przypisujemy im kolumny z wartościami binarnymi i usuwamy kolumnę Marka pojazdu
df_filtered["Segment_marki"] = df_filtered["Marka_pojazdu"].apply(categorize_brand)
df_filtered = pd.get_dummies(df_filtered, columns=["Segment_marki"], dtype=int)
df_filtered = df_filtered.drop(columns=["Marka_pojazdu"])


# Uzupełniamy medianą wartości numeryczne
df_filtered["Przebieg_km"].fillna(df_filtered["Przebieg_km"].median(), inplace=True)
df_filtered["Rok_produkcji"].fillna(df_filtered["Rok_produkcji"].median(), inplace=True)
df_filtered["Moc_KM"].fillna(df_filtered["Moc_KM"].median(), inplace=True)
df_filtered["Pojemnosc_cm3"].fillna(df_filtered["Pojemnosc_cm3"].median(), inplace=True)

# Zamieniamy wartości object na liczby
df_filtered["Rodzaj_paliwa"] = df_filtered["Rodzaj_paliwa"].fillna("unknown").astype("category").cat.codes
df_filtered["Naped"] = df_filtered["Naped"].fillna("unknown").astype("category").cat.codes
df_filtered["Skrzynia_biegow"] = df_filtered["Skrzynia_biegow"].fillna("unknown").astype("category").cat.codes
df_filtered["Typ_nadwozia"] = df_filtered["Typ_nadwozia"].fillna("unknown").astype("category").cat.codes
df_filtered["Kolor"] = df_filtered["Kolor"].fillna("unknown").astype("category").cat.codes
df_filtered["Model_pojazdu"] = df_filtered["Model_pojazdu"].fillna("unknown").astype("category").cat.codes

# Dla każdego segmentu (Budżetowe, Średnie, Premium) liczymy średnią liczbę wyposażenia.
# Samochody z większą liczbą opcji niż średnia → oznaczamy jako + (lepiej wyposażone).
# Samochody z mniejszą liczbą opcji niż średnia → oznaczamy jako - (słabiej wyposażone).
# Przekształcamy każdą listę na liczbę dostępnych opcji:
df_filtered["Liczba_wyposazenia"] = df_filtered["Wyposazenie"].apply(lambda x: len(str(x).split(", ")))

# Teraz liczymy średnią liczbę wyposażenia dla Premium, Średnie, Budżetowe:
srednie_wyposazenie = df_filtered.groupby(["Segment_marki_Budżetowe", "Segment_marki_Średnie", "Segment_marki_Premium"])["Liczba_wyposazenia"].mean()

# Przypisujemy auta do dwóch grup - z + lub z -
def oznacz_wyposazenie(row):
    if row["Segment_marki_Budżetowe"] == 1:
        srednia = srednie_wyposazenie["Segment_marki_Budżetowe"]
    elif row["Segment_marki_Średnie"] == 1:
        srednia = srednie_wyposazenie["Segment_marki_Średnie"]
    elif row["Segment_marki_Premium"] == 1:
        srednia = srednie_wyposazenie["Segment_marki_Premium"]
    else:
        return "-"  # Jeśli z jakiegoś powodu nie ma segmentu, domyślnie "-"
    
    return "+" if row["Liczba_wyposazenia"] > srednia else "-"

srednie_wyposazenie = {
    "Segment_marki_Budżetowe": df_filtered[df_filtered["Segment_marki_Budżetowe"] == 1]["Liczba_wyposazenia"].mean(),
    "Segment_marki_Średnie": df_filtered[df_filtered["Segment_marki_Średnie"] == 1]["Liczba_wyposazenia"].mean(),
    "Segment_marki_Premium": df_filtered[df_filtered["Segment_marki_Premium"] == 1]["Liczba_wyposazenia"].mean()
}


df_filtered["Wyposazenie_kategoria"] = df_filtered.apply(oznacz_wyposazenie, axis=1)
df_filtered["Wyposazenie_kategoria"] = df_filtered["Wyposazenie_kategoria"].map({"+": 1, "-": 0})
df_filtered = df_filtered.drop(columns=["Wyposazenie"])

# Zapisujemy do trenowania
df_filtered.to_csv("../data/cleaned_train.csv", index=False)

/var/folders/02/2thd89_s1ln77brcp9tybl4h0000gn/T/ipykernel_19305/619514072.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered["Przebieg_km"].fillna(df_filtered["Przebieg_km"].median(), inplace=True)
/var/folders/02/2thd89_s1ln77brcp9tybl4h0000gn/T/ipykernel_19305/619514072.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object